# Window functions in SQL

We are going to look at the concept of _window fucntions_ in SQL. The basic idea is we use a window function when we need to use information that is not in the current row to calculate the current row. 

We will show how to use the same type of problems using
- pandas
- group by and joins in SQL
- subselects in SQL
- ... and finally window functions

## Getting the data in df and postgres

In [ ]:
import pandas as pd

df = pd.read_csv('salaries.csv')

df

In [ ]:
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

params = {
    'host': '18.220.115.81',
    'user': 'ubuntu',
    'port': 5432
}

# Connect and create database, disconnect, and reconnect to the right database
connection = connect(**params, dbname='ubuntu')
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
connection.cursor().execute('CREATE DATABASE store;')
connection.close()

In [ ]:
# This is a different way of creating tables
# It is convinient if there are many columns
from sqlalchemy import create_engine
connection_string = f'postgres://ubuntu:{params["host"]}@{params["host"]}:{params["port"]}/store'
engine = create_engine(connection_string)
df.to_sql('sale', engine, index=False)

In [ ]:
connection = connect(**params, dbname='store')
cursor = connection.cursor()
cursor.execute("SELECT * FROM sale;")
cursor.fetchall()

## Question: For each person, give the salary as a fraction of the highest paid person in that department.

Note that each row contains the salary and department, but it _doesn't_ contain the highest pay in that category. We need to look at other rows. Let's start by looking for the highest pay per department:

### Method 1: joining with DF

In [ ]:
highest_dept = df.groupby('dept').salary.max()
highest_dept = pd.DataFrame(highest_dept).reset_index().rename(columns={'salary': 'highest_salary_in_dept'})
highest_dept

Here is the long way: 
1. `Groupby` to get the aggregate information in a new series / df
2. Join on `dept` to get the max salary
3. Then manpiulate columns

In [ ]:
# merges on the columns that have the same name (in this case, dept)
df_merge1 = pd.merge(df,highest_dept)
df_merge1

In [ ]:
df_merge1['fraction_of_highest'] = (df_merge1['salary'] / 
                                    df_merge1['highest_salary_in_dept'])
df_merge1

#### Exercise 1:

Give each person's salary as a fraction of then _entire_ department salary budget
(e.g. employee1 has a salary of 5000, and the sales department has a total salary budget of 5000 + 4800 + 7800 = 14500, so we would expect this row to have 5000/14500 = 0.345)

### Method 2: Using pandas `transform`

In [ ]:
df2 = df.copy()
df2

In [ ]:
# Transform:
# Apply a function to a grouped series, and return the result for the group to each row
df2.groupby('dept').salary.transform(max)

In [ ]:
# Save the result, and write using a lambda function for clarity
df2['highest_salary_in_dept'] = df2.groupby('dept').salary.transform(lambda dept: max(dept))

In [ ]:
df2

In [ ]:
# now get the fraction
df2['fraction_of_highest'] = (df2['salary'] / 
                              df2['highest_salary_in_dept'])

df2

#### Exercise 2

Redo exercise 1, giving each person's salary as a fraction of then entire department salary budget, using this new method

## Method 3: GROUPBY/JOIN in SQL:

In [ ]:
query = """
WITH dept_info AS (
  SELECT dept, max(salary) AS max_salary, sum(salary) as salary_budget FROM sale
  GROUP BY dept
)

SELECT sale.*, max_salary, CAST(salary AS DOUBLE PRECISION)/max_salary as max_frac  
  FROM sale LEFT JOIN dept_info ON sale.dept = dept_info.dept;
"""

cursor.execute(query)
cursor.fetchall()

### Method 4: Using a window function

A window function are limited in number, and are used to generate a column (i.e. they occur between `SELECT` and `FROM`). They are of the form
```sql
...
  function OVER (PARTION BY ..... ORDER BY ......[ACS or DESC])
```
where `function` is the _window function_. We perform aggregations on the records that are inside the window.

* `PARTITION` is tells you what is included in the window
* `ORDER BY` tells us the order that appears in the window. For `max`, this doesn't matter (but we will see some examples where if does)

In [ ]:
# Example of max used
query="""
  SELECT *, max(salary) OVER (PARTITION BY dept) FROM sale;
"""

cursor.execute(query)
cursor.fetchall()

In [ ]:
# Can also solve the problem
query="""
  SELECT *, 
        CAST(salary AS DOUBLE PRECISION)/max(salary) OVER (PARTITION BY dept) AS fraction_of_max
        FROM sale;
"""

cursor.execute(query)
cursor.fetchall()

If the window makes the function long, we can make the window as an alias

In [ ]:
# same query again:
query = """
SELECT *, CAST(salary AS DOUBLE PRECISION)/max(salary) OVER my_window FROM SALE
  WINDOW my_window AS (PARTITION BY dept)
"""

cursor.execute(query)
cursor.fetchall()

## Other examples of window functions

We can also use window fucntions that are cumulative. 

### rank()

Let's look at the top 3 earners in each category

In [ ]:
# first attempt: get the ranks
query = """
SELECT *, rank() OVER my_window as the_rank FROM sale
  WINDOW my_window AS (PARTITION BY dept ORDER BY salary DESC)
"""

cursor.execute(query)
cursor.fetchall()

In [ ]:
# This fails .... where can be tricky. Column "the_rank" doesn't exist yet
query = """
rollback;
SELECT *, rank() OVER my_window as the_rank FROM sale
  WINDOW my_window AS (PARTITION BY dept ORDER BY salary DESC)
  WHERE the_rank < 3;
"""

cursor.execute(query)
cursor.fetchall()

In [ ]:
# fix with a subquery
query = """
rollback;
SELECT * FROM
 (SELECT *, rank() OVER my_window as the_rank FROM sale
      WINDOW my_window AS (PARTITION BY dept ORDER BY salary DESC)
  ) AS ordered
  WHERE ordered.the_rank < 3;
"""

cursor.execute(query)
cursor.fetchall()

### cumsum()

Let's look at the cumulative sum of salary, from smallest to largest

In [ ]:
query = """
rollback;
SELECT *, sum(salary) OVER my_window FROM sale
    WINDOW my_window AS (PARTITION BY dept ORDER BY salary DESC)
"""

cursor.execute(query)
cursor.fetchall()

In [ ]:
query = """
rollback;
SELECT *, CASE WHEN dept='sales' THEN 1 ELSE 0 END AS front_line FROM sale;
"""

cursor.execute(query)
cursor.fetchall()